<a href="https://colab.research.google.com/github/tranquockinh/Fast-Inversion/blob/main/Fast_Inversion_Forward_Calculation(1-9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Earth model assumption: ground truth generation**
### (1) Creating a shear wave velocity profile
### (2) Computing the particle displacement of vertucal component



In [ ]:
# Earth proflie
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
LayerData = np.array([[100,0],
                      [200,2],
                      [250,5],
                      [400,10],
                      [400,np.inf]])
z = LayerData[:,1]
num_layer = len(z)-1
Vs = LayerData[:,0]
Layer_Vs = Vs[0:-1]
fig,ax = plt.subplots(1,2,figsize=(10,5),dpi=100)
#fig,ax = plt.subplots(figsize=(4,6),dpi=100)
ax[0].step(LayerData[:,0],LayerData[:,1])
ax[0].invert_yaxis()
xlimit = [0,LayerData[-1,0]+50]
#plt.xlim(xlimit[0],xlimit[1])
for i in range(len(LayerData[:,0])-1):
    ax[0].hlines(y=LayerData[i,1],xmin=xlimit[0],xmax=xlimit[1],linestyle='--',linewidth=0.5)
ax[0].set_xlabel('Shear wave velocity, Vs [m/s]')
ax[0].set_ylabel('Depth, z [m]')
ax[0].xaxis.set_label_position('top')
ax[0].xaxis.tick_top()
mid_layer = LayerData[:-1,1]+0.5*(LayerData[1:,1]-LayerData[:-1,1])
label = LayerData[:-1,0]
for i in range(num_layer):
  layer = ('Layer {}: Vs = {}').format(i+1,label[i])
  ax[0].annotate(layer, (label[i],mid_layer[i]), textcoords='offset points', xytext=(10,0))
halfspace_Vs = ('Layer {} Halfspace Vs = {}').format(num_layer,LayerData[-1,0],0)
ax[0].annotate(halfspace_Vs,(LayerData[-3,0],LayerData[-2,1]),textcoords='offset points',horizontalalignment='center', xytext=(0,-20))
ax[0].spines['bottom'].set_color('white')
ax[0].spines['right'].set_color('white')
# Wavelength data
Lambda = np.zeros((25))
DeltaLambda = 1
for i in range(len(Lambda)):
    Lambda[0] = 2 # m
    if i > 0:
        Lambda[i] = Lambda[i-1] + DeltaLambda
  # Plot particle motion curve
def Partical_displacement(WL,z,num_layer):
    SF=1
    cv = np.array([0.2507, -0.4341, -0.8474*2*np.pi, -0.3933*2*np.pi])
    cv1 = cv[0] 
    cv2 = cv[1] 
    cv3 = cv[2] 
    cv4 = cv[3]
    # Normalization
    zn = z/WL
    # compute amplitude of particle displacements
    pmv = (cv1*np.exp(cv3*zn) + cv2*np.exp(cv4*zn))*SF
    pmv_norm = pmv/pmv[0]
    return pmv,pmv_norm,zn
pmv,pmv_norm,zn = Partical_displacement(Lambda[10],z,num_layer)
# plotting
ax[1].plot(pmv_norm,zn,'-or', markerfacecolor='None')
ax[1].xaxis.tick_top()
ax[1].xaxis.set_label_position('top')
ax[1].set_xlabel('Normalized vertical particle motion \n at a wavelength, $wp/w_{p0}$ [mm]')
ax[1].set_ylabel('Normalized depth, zn [m]')
ax[1].vlines(x=0,ymin=zn[0],ymax=zn[-1],linestyles='--',linewidth=1)
ax[1].legend(['PR=0.5'],loc='lower right')
ax[1].invert_yaxis()
for i in range(num_layer):
  label = ('(Layer {}): {}').format(i+1,np.round(pmv_norm[i],4))
  ax[1].annotate(label, (pmv_norm[i],zn[i]), textcoords='offset points', xytext=(20,0))
ax[1].spines['bottom'].set_color('white')
ax[1].spines['right'].set_color('white')

In [ ]:
'''
fig,ax = plt.subplots(figsize=(4,6),dpi=100)
def plot_sublayers():
  num_sublayer = np.linspace(LayerData[0,1],LayerData[-2,1],5*(num_layer-1))
  num_sublayer = np.append(num_sublayer,np.inf)
  sublayers = len(num_sublayer)
  pmv,pmv_norm,zn = Partical_displacement(Lambda[10],num_sublayer,sublayers)
  ax.plot(pmv_norm,zn,'-or', markerfacecolor='None')
  ax.xaxis.tick_top()
  ax.xaxis.set_label_position('top')
  ax.set_xlabel('Normalized vertical particle motion \n at a wavelength, $wp/w_{p0}$ [mm]')
  ax.set_ylabel('Normalized depth, zn [m]')
  ax.vlines(x=0,ymin=zn[0],ymax=zn[-1],linestyles='--',linewidth=1)
  ax.legend(['PR=0.5'],loc='lower right')
  ax.invert_yaxis()
  ax.spines['bottom'].set_color('white')
  ax.spines['right'].set_color('white')
  for i in range(sublayers):
    label = ('(Sub-Layer {}): {}').format(i+1,np.round(pmv_norm[i],4))
    ax.annotate(label, (pmv_norm[i],zn[i]), textcoords='offset points', xytext=(20,0))
  for i in range(len(LayerData[:,0])-1):
    ax.hlines(y=LayerData[i,1]/Lambda[10],xmin=0,xmax=1,linestyle='--',linewidth=0.5)
  return num_sublayer
num_sublayer = plot_sublayers()
'''

## Look at the particle displacement curve

## Computing normaized weighting factor
### Compute with each value of supposed wavelength 

In [ ]:
def Compute_Weights(WL,z):
    num_layer = len(z)-1
    Ai = np.zeros((num_layer,1),dtype='object')
    wp = np.zeros((num_layer,1),dtype='float')
    SF = 1
    low_bound = z[:-1]
    up_bound = z[1:]
    h = sp.symbols('h')
    cv = np.array([0.2507, -0.4341, -0.8474*2*np.pi, -0.3933*2*np.pi])
    cv1 = cv[0] 
    cv2 = cv[1] 
    cv3 = cv[2] 
    cv4 = cv[3]
    pmv_norm = (cv1*sp.exp(cv3/WL*h) + cv2*sp.exp(cv4/WL*h))*SF 
    for j in range(num_layer):
        Ai[j,0] = sp.integrate(pmv_norm,(h,low_bound[j],up_bound[j]))
        A = sp.integrate(pmv_norm,(h,low_bound[0],up_bound[-1])) 
        wp[j,0] = Ai[j] / A
    return wp
wp = np.zeros((len(Lambda)),dtype='object')
for i in range(len(wp)):
    wp[i] = np.zeros((len(wp)))
    wp[i] = Compute_Weights(Lambda[i],z)
    print(('WAVELENGTH {} PASSED').format(i+1))
    ERROR = 1E-6
    if (np.sum(wp[i]) - 1 > ERROR):
        print('NOT PASSED')
    else: continue

In [ ]:
# Distribution of weighting
labels = ['Layer 1','Layer 2','Layer 3','Layer 4']
fig,ax=plt.subplots(figsize=(4,6),dpi=100)
plt.plot(wp[1],labels,'-o',markerfacecolor='None')
plt.xlabel('Weighting average factor, $w_{p}$ \n at a wavelength')
plt.ylabel('Layers')
ax.invert_yaxis()
ax.xaxis.set_label_position('top')
ax.xaxis.tick_top()
for i in range(num_layer):
      label = ('{}').format(np.round(wp[1][i],4))
      ax.annotate(label, (wp[1][i],labels[i]), textcoords='offset points', xytext=(10,-10))
ax.spines['bottom'].set_color('white')
ax.spines['right'].set_color('white')

In [ ]:
# Compute phase velocity
def Phase_velocity(Layer_Vs,wp):
    Vph = np.dot(0.93*Layer_Vs,wp)
    return Vph
Vph = np.zeros((len(Lambda)))
for i in range(len(Lambda)):
    Vph[i] = Phase_velocity(Layer_Vs,wp[i])
print(Vph)
fig,ax=plt.subplots(figsize=(4,6),dpi=100)
ax.plot(Vph,Lambda,'-o',markerfacecolor='None')
ax.invert_yaxis()
ax.set_xlabel('Phase velocity, Vph [m/s]')
ax.xaxis.set_label_position('top')
ax.xaxis.tick_top()
ax.set_ylabel('Wavelength, [m]')

# Inversion 

## Compute weights

In [ ]:
def inversionData(Vph,Lambda):
  Con_Depth = 0.35 * Lambda
  Con_Vs = (1/0.93) * Vph
  wp = np.zeros((len(Lambda)),dtype='object')
  for i in range(len(wp)):
      wp[i] = np.zeros((num_layer)) # ROOTS
      wp[i] = Compute_Weights(Lambda[i],Con_Depth)
      print(('WAVELENGTH {} PASSED').format(i+1))
      ERROR = 1E-6
      if (np.sum(wp[i]) - 1 > ERROR):
          print('NOT PASSED')
      else: continue
  return wp,Con_Vs
wp,Con_Vs = inversionData(Vph,Lambda)

In [ ]:
fig,ax=plt.subplots(figsize=(4,6),dpi=100)
Layer = np.zeros((len(wp[0])),dtype='object')
for i in range(len(wp[0])):
  Layer[i] = ('Layer {}').format(i+1)
plt.plot(wp[1],Layer,'-o',markerfacecolor='None')
ax.invert_yaxis()
ax.xaxis.set_label_position('top')
ax.xaxis.tick_top()


## Compute phase velocity

In [ ]:
# Compute phase velocity
weighting_factor = np.zeros((len(Con_Vs)),dtype='object')
Vs = np.zeros((len(Vph),1))
disperse = np.zeros((len(Vph)))
for i in range(len(Con_Vs)):
  weighting_factor[i] = np.zeros((len(wp[i])))
  if i == 0:
    weighting_factor[i] = wp[i]

  else: 
    weighting_factor[i] = wp[i]
    weighting_factor[i][:i] = weighting_factor[i-1][:i]
  # Vs i
  Vs[i] = (1/0.93) * Vph[i]
  Vs[i:] = Vs[i]
  disperse[i] = np.dot(wp[i].flatten(),Vs[1:].flatten())
  
print(disperse)